In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# get tfidf

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/IfKnowDebtor/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(4560, 2)


,label,split_text
0,1,无声
1,0,他 失去 了 他 的 手机
2,2,今天 的 天气 很 好
3,2,你 在 做 什么 ？
4,2,喜喜
5,0,她 是 我 的 老板
6,1,你 正在 寻找 错误 的 人
7,0,多年 的 朋友 嘿 ？
8,0,我 是 他 的 母亲
9,0,她 是 我 的 老板


In [3]:
# phrase_vectorizer1 = TfidfVectorizer(ngram_range=(1,3),
#                                     strip_accents='unicode', 
#                                     max_features=100000, 
#                                     analyzer='word',
#                                     sublinear_tf=True,
#                                     token_pattern=r'\w{1,}')
# data.split_text.iloc[:2]
# dd = ['I like apple do you like it', 'I do not like banana']
# phrase_vectorizer1.fit(dd)

# print('transform phrase')
# phrase = phrase_vectorizer1.transform(dd)


In [4]:
data.split_text.iloc[1]

'他 失去 了 他 的 手机'

In [5]:
# phrase_vectorizer1.vocabulary_

In [6]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<4560x6378 sparse matrix of type '<class 'numpy.float64'>'
	with 43480 stored elements in Compressed Sparse Row format>

# SVC

In [7]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.964035087719


In [8]:
print(clf.predict(phrase))

[2 0 2 ..., 0 1 0]


In [9]:
import pickle
pickle.dump(clf, open("../../savedModel/IDClassifier/l_svc.pickle", "wb"))

In [10]:
aa = pickle.load(open("../../savedModel/IDClassifier/l_svc.pickle", 'rb'))

In [11]:
aa.predict(phrase)

array([2, 0, 2, ..., 0, 1, 0])

# logistic

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.948245614035


In [14]:
print(log_r.predict(phrase))

[2 0 2 ..., 0 1 0]


In [15]:
# l_svc.predict(phrase)-log_r.predict(phrase)

# lightGBM

In [16]:
import lightgbm as lgb

In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [18]:
onelabels

array([1, 0, 2, ..., 0, 1, 0])

In [19]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [20]:
lables

array([[1, 1, 1]])

In [21]:
data.label.values

array([1, 0, 2, ..., 0, 1, 0])

In [22]:
# lables.todense()

In [23]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.140132
[10]	valid_0's multi_error: 0.124342
[15]	valid_0's multi_error: 0.105044
[20]	valid_0's multi_error: 0.0997807
[25]	valid_0's multi_error: 0.0942982
[30]	valid_0's multi_error: 0.0916667
[35]	valid_0's multi_error: 0.0894737
[40]	valid_0's multi_error: 0.0848684
[45]	valid_0's multi_error: 0.0826754
[50]	valid_0's multi_error: 0.0807018
[55]	valid_0's multi_error: 0.0789474
[60]	valid_0's multi_error: 0.077193
[65]	valid_0's multi_error: 0.0767544
[70]	valid_0's multi_error: 0.0754386
[75]	valid_0's multi_error: 0.0741228
[80]	valid_0's multi_error: 0.0732456
[85]	valid_0's multi_error: 0.0719298
[90]	valid_0's multi_error: 0.0714912
[95]	valid_0's multi_error: 0.0701754
[100]	valid_0's multi_error: 0.0703947
[105]	valid_0's multi_error: 0.0701754
[110]	valid_0's multi_error: 0.0701754
[115]	valid_0's multi_error: 0.0692982
[120]	valid_0's multi_error: 0.0688596
[125]	valid_0's multi_err

In [24]:
print(lgbm_model.predict(phrase))

[[  1.55703183e-01   1.73302938e-01   5.97600071e-01]
 [  9.91624632e-01   8.68265568e-03   2.84662200e-03]
 [  1.60366043e-03   8.37586804e-03   9.97496613e-01]
 ..., 
 [  9.99854726e-01   1.82644882e-04   5.66295825e-05]
 [  5.55430383e-03   9.57718833e-01   3.30640729e-02]
 [  9.99842542e-01   1.79785553e-04   5.66295825e-05]]


In [25]:
# lgbm_model.predict(phrase)

In [26]:
phrase

<4560x6378 sparse matrix of type '<class 'numpy.float64'>'
	with 43480 stored elements in Compressed Sparse Row format>

# Save

In [27]:
import pickle

pickle.dump(phrase_vectorizer, open("../../savedModel/IfKnowDebtor/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))


# save linear svc
pickle.dump(clf, open("../../savedModel/IfKnowDebtor/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/IfKnowDebtor/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/IfKnowDebtor/Lgbm.pickle", "wb"))

In [28]:
pickle.load(open("../../savedModel/IfKnowDebtor/Logistic.pickle", 'rb'))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
# test = phrase_vectorizer.transform(['你 说 呢'])
test = phrase_vectorizer.transform(['认 不 到'])


In [30]:
test

<1x6378 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [31]:
clf.predict_proba(test) # linear svc

array([[ 0.03675858,  0.91715486,  0.04608656]])

In [32]:
log_r.predict_proba(test) # logistic

array([[ 0.13180316,  0.62076309,  0.24743375]])

In [33]:
lgbm_model.predict(test) # light gbm

array([[ 0.00132372,  0.90017848,  0.11386062]])

# labeling

In [34]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.917154859779


# save the whole thing

In [35]:
# import jieba
# import numpy as np

# class IfKnowDebtor:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred)<threshold:
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [36]:
%load_ext autoreload
%autoreload 2

In [1]:
import pickle
clf = pickle.load(open("../../savedModel/IfKnowDebtor/LinearSVC.pickle", "rb"))
lgbm_model = pickle.load(open("../../savedModel/IfKnowDebtor/Lgbm.pickle", "rb"))
phrase_vectorizer = pickle.load(open("../../savedModel/IfKnowDebtor/tfidf.pickle", "rb"))
log_r = pickle.load(open("../../savedModel/IfKnowDebtor/Logistic.pickle", "rb"))

In [2]:
from IfKnowDebtor_py import IfKnowDebtor
ifk = IfKnowDebtor(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer,) #jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/IfKnowDebtor/IfKnowDebtor.pickle", "wb"))